Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [1]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss

import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,


In [2]:
path = "../"
files = glob2.glob(f'{path}real_data/*.h5')
files = [f[len(f"'{path}real_data"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 '10X_PBMC_select_2100',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'worm_neuron_cell_select_2100',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'mouse_ES_cell_select_2100',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 'mouse_bladder_cell_select_2100',
 '10X_PBMC']

In [3]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [4]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_1model.pkl")

In [9]:
# df = pd.DataFrame()

for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    cluster_number = np.unique(Y).shape[0]
    nb_genes = 1500
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])

    X = train.preprocess(X, nb_genes=nb_genes)
    for temperature in tqdm([ 0.01,
                             # 0.07, 0.5,
                            ]):
        for dropout in [ 0.9,
#                         0.8, 0.2, 0.5 
                       ]:
            for lr in [1e-5
                       #, 1e-4, 1e-6
                      ]:
                for layers in [
                    [15], [30]
                   # [100], [200]
#                     [50],[100, 30],  [200, 100, 30],  [200, 100, 30, 30]
                              ]:
                    for run in range(2):
                        torch.manual_seed(run)
                        torch.cuda.manual_seed_all(run)
                        np.random.seed(run)
                        random.seed(run)
                        torch.backends.cudnn.deterministic = True
                        torch.backends.cudnn.benchmark = False
                        dresults = train.run(X,
                                             cluster_number,
                                             dataset,
                                             Y=Y,
                                             nb_epochs=20,
                                             lr=lr,
                                             temperature=temperature,
                                             dropout=dropout,
                                             evaluate=False,
                                             n_ensemble=1,
                                             layers=layers,
                                             save_to=f"{path}output/real_data/{run}/",
                                             save_pred = False)
                        dresults["temperature"] = temperature
                        dresults["dropout"] = dropout
                        dresults["lr"] = lr
                        dresults["layers"] = str(layers)
                        dresults["run"] = run
                        print(f".", end = "")
#                         print(f"# {temperature}, {dropout}, {lr}, {layers}", 
#                               dresults.get('COMBINED_kmeans_ari', ""),
#                               dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
#                               dresults.get('leiden_ari_0', ""))
                        df = df.append(dresults, ignore_index=True)

                        df.to_pickle(f"{path}output/pickle_results/real_data/real_data_1model.pkl")

>>>>> Data Quake_Smart-seq2_Trachea
SCZI  0.8291128219663276
(1350, 23341) (1350, 23341) keeping 1500 genes


....
>>>>> Data Quake_Smart-seq2_Diaphragm
SCZI  0.9596701189611787
(870, 23341) (870, 23341) keeping 1500 genes


....
>>>>> Data Quake_10x_Spleen
SCZI  0.909534824400645
(9552, 23341) (9552, 23341) keeping 1500 genes


....
>>>>> Data Young
SCZI  0.6629811886776039
(5685, 33658) (5685, 33658) keeping 1500 genes


....
>>>>> Data 10X_PBMC_select_2100
SCZI  0.597949963742046
(2100, 16653) (2100, 16653) keeping 1500 genes


...

....
>>>>> Data Quake_Smart-seq2_Limb_Muscle
SCZI  0.9736759009464299
(1090, 23341) (1090, 23341) keeping 1500 genes


....
>>>>> Data Muraro
SCZI  0.7246271034758398
(2122, 19046) (2122, 19046) keeping 1500 genes


....
>>>>> Data mouse_bladder_cell_select_2100
SCZI  0.521167400509594
(2100, 20670) (2100, 20670) keeping 1500 genes


....
>>>>> Data 10X_PBMC
SCZI  0.5845649506030623
(4271, 16653) (4271, 16653) keeping 1500 genes


....


In [11]:
df = pd.read_pickle(f"../output/pickle_results/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"])

layers              [100, 30]                         [100]     [15]  \
lr                   0.000001  0.000010  0.000100  0.000010 0.000010   
temperature dropout                                                    
0.01        0.2      0.511959  0.511959  0.511959       NaN      NaN   
            0.5      0.561639  0.561639  0.561639       NaN      NaN   
            0.8      0.638963  0.638963  0.638963       NaN      NaN   
            0.9      0.627648  0.627648  0.627648  0.654767  0.64641   
0.07        0.2      0.277751  0.277751  0.277751       NaN      NaN   
            0.5      0.484438  0.484438  0.484438       NaN      NaN   
            0.8      0.618134  0.618134  0.618134       NaN      NaN   
            0.9      0.648517  0.648517  0.648517       NaN      NaN   
0.50        0.2      0.433748  0.433748  0.433748       NaN      NaN   
            0.5      0.493075  0.493075  0.493075       NaN      NaN   
            0.8      0.571350  0.571350  0.571350       NaN      NaN   
            0.9      0.632408  0.632408  0.632408       NaN      NaN   

layers              [200, 100, 30, 30]                     [200, 100, 30]  \
lr                            0.000001  0.000010  0.000100       0.000001   
temperature dropout                                                         
0.01        0.2               0.449366  0.449366  0.449366       0.472721   
            0.5               0.530522  0.530522  0.530522       0.522939   
            0.8               0.625792  0.625792  0.625792       0.637956   
            0.9               0.602669  0.602669  0.602669       0.623157   
0.07        0.2               0.228623  0.228623  0.228623       0.236237   
            0.5               0.422648  0.422648  0.422648       0.438549   
            0.8               0.636183  0.636183  0.636183       0.634162   
            0.9               0.654658  0.654658  0.654658       0.635984   
0.50        0.2               0.401195  0.401195  0.401195       0.430879   
            0.5               0.468168  0.468168  0.468168       0.501659   
            0.8               0.562717  0.562717  0.562717       0.574581   
            0.9               0.634288  0.634288  0.634288       0.624716   

layers                                      [200]      [30]      [50]  \
lr                   0.000010  0.000100  0.000010  0.000010  0.000001   
temperature dropout                                                     
0.01        0.2      0.472721  0.472721       NaN       NaN  0.555164   
            0.5      0.522939  0.522939       NaN       NaN  0.604148   
            0.8      0.637956  0.637956       NaN       NaN  0.673766   
            0.9      0.623157  0.623157  0.656382  0.677728  0.692151   
0.07        0.2      0.236237  0.236237       NaN       NaN  0.320988   
            0.5      0.438549  0.438549       NaN       NaN  0.506122   
            0.8      0.634162  0.634162       NaN       NaN  0.598195   
            0.9      0.635984  0.635984       NaN       NaN  0.652793   
0.50        0.2      0.430879  0.430879       NaN       NaN  0.529359   
            0.5      0.501659  0.501659       NaN       NaN  0.573374   
            0.8      0.574581  0.574581       NaN       NaN  0.641304   
            0.9      0.624716  0.624716       NaN       NaN  0.665398   

layers                                   
lr                   0.000010  0.000100  
temperature dropout                      
0.01        0.2      0.555164  0.555164  
            0.5      0.604148  0.604148  
            0.8      0.673766  0.673766  
            0.9      0.692151  0.692151  
0.07        0.2      0.320988  0.320988  
            0.5      0.506122  0.506122  
            0.8      0.598195  0.598195  
            0.9      0.652793  0.652793  
0.50        0.2      0.529359  0.529359  
            0.5      0.573374  0.573374  
            0.8      0.641304  0.641304  
            0.9      0.665398  0.665398

In [12]:
df = pd.read_pickle(f"../output/pickle_results/real_data_1model.pkl")
df.groupby(["temperature", "layers", "dropout", "lr"])["kmeans_ari_0"].mean().unstack(["layers", "lr"]).max()

layers              lr      
[100, 30]           0.000001    0.648517
                    0.000010    0.648517
                    0.000100    0.648517
[100]               0.000010    0.654767
[15]                0.000010    0.646410
[200, 100, 30, 30]  0.000001    0.654658
                    0.000010    0.654658
                    0.000100    0.654658
[200, 100, 30]      0.000001    0.637956
                    0.000010    0.637956
                    0.000100    0.637956
[200]               0.000010    0.656382
[30]                0.000010    0.677728
[50]                0.000001    0.692151
                    0.000010    0.692151
                    0.000100    0.692151
dtype: float64

In [ ]:
df.groupby("dataset").mean()

# Importance of input size

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_genes", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    for run in range(2):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        for nb_genes in [100, 200, 500, 1000, 1500, 2000, 5000, 8000]:

            X = np.array(data_mat['X'])
            Y = np.array(data_mat['Y'])
            perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
            print(f"Perc 0 {perc_0}")
            cluster_number = np.unique(Y).shape[0]

            X = train.preprocess(X, nb_genes=nb_genes)
            nb_zeros = int(0.8 * nb_genes)
            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=100,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/inputs/{dataset}_{nb_genes}/")

            #         df.loc[df.shape[0]] = [
            #                 dataset, perc_0, nb_genes, 'kmeans_representation_0',dresults['kmeans_representation_0']
            #             ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, nb_genes, 'leiden_representation_0',
                dresults['leiden_representation_0'], run]

#             pxt = PCA(2).fit_transform(X)
#             dresults["original"] = utils.evaluate(X, Y, cluster_number)[1]
#             dresults["pca"] = utils.evaluate(pxt, Y, cluster_number)[1]
            print(dresults)
    df.to_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_input_size.pkl")

In [ ]:
df.groupby(["nb_genes"]).mean()

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_genes": "Nb input genes"})

In [ ]:
df["dataset"].unique()


In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size.pdf", bbox_inches='tight')

In [ ]:
datasets = ['10X PBMC (2100)',
       'Mouse ES\nCell (2100)', 'Worm Neuron\nCell (2100)',
       'Mouse Bladder\n Cell (2100)']
plt.figure(figsize=(10, 3.3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb input genes",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title= "Nb input genes",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_input_size_2100.pdf", bbox_inches='tight')

# Importance of the number of epochs

In [ ]:
df = pd.DataFrame(
    columns=["dataset", "perc0", "nb_epochs", "exp", "ari", "run"])
print(df.shape)
for dataset in files:

    print(f">>>>> Data {dataset}")

    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    nb_genes = 1500
    for epochs in [5, 50, 100, 300]:

        X = np.array(data_mat['X'])
        Y = np.array(data_mat['Y'])
        perc_0 = np.where(X == 0)[0].shape[0] / (X.shape[0] * X.shape[1])
        print(f"Perc 0 {perc_0}")
        cluster_number = np.unique(Y).shape[0]

        X = train.preprocess(X, nb_genes=nb_genes)
        nb_zeros = int(0.8 * nb_genes)
        for run in range(2):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

            dresults = train.train(
                X,
                cluster_number,
                dataset,
                Y,
                n_ensemble=1,
                epochs=epochs,
                nb_zeros=nb_zeros,
                save_to=f"{path}output/real_data/epochs/{dataset}_{epochs}/")

            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'kmeans_representation_0',
                dresults['kmeans_representation_0'], run
            ]
            df.loc[df.shape[0]] = [
                dataset, perc_0, epochs, 'leiden_representation_0',
                dresults['leiden_representation_0'], run
            ]

            print(dresults)
            df.to_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
df = pd.read_pickle(f"{path}output/pickle_results/real_data_epochs.pkl")

In [ ]:
dataset_names = {
    '10X_PBMC': '10X PBMC',
    '10X_PBMC_select_2100': '10X PBMC (2100)',
    'mouse_ES_cell': 'Mouse ES\nCell',
    'mouse_ES_cell_select_2100': 'Mouse ES\nCell (2100)',
    'worm_neuron_cell_select_2100': 'Worm Neuron\nCell (2100)',
    'worm_neuron_cell': 'Worm Neuron\nCell',
    'mouse_bladder_cell': 'Mouse Bladder\nCell',
    'mouse_bladder_cell_select_2100': 'Mouse Bladder\n Cell (2100)'
}

df["dataset"] = df["dataset"].apply(lambda x: dataset_names[x])

df = df.rename(columns = {"nb_epochs": "Nb epochs"})

In [ ]:
import seaborn as sns
datasets = ['10X PBMC',  'Mouse ES\nCell','Worm Neuron\nCell', 'Mouse Bladder\nCell']
plt.figure(figsize=(7, 3))
ax = plt.subplot(111)
sns.barplot(
    hue="Nb epochs",
    y="ari",
    x="dataset",
    data=df[df["dataset"].isin(datasets)],
    ax=ax,
    edgecolor='black',
    linewidth=1.5,
)
plt.ylabel("ARI")
plt.xlabel("")
plt.legend(title ="Number of epochs",bbox_to_anchor=(1, 1))
sns.despine()
plt.tight_layout()

plt.savefig(f"{path}diagrams/real_nb_epochs.pdf", bbox_inches='tight')